In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Github/sports-article-classification/
%pip install slackclient
%pip install doccano-client
#%pip install -e .
%pip install wandb
%pip install scikit-multilearn
%pip install scikit-optimize
%cd src/modelling

%cd ../..
%pip install -e .

/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 6.5 MB/s eta 0:00:00
  Attempting unins

In [3]:
#%cd ../..
#%pip install -e .

In [4]:
from src.data.preprocessing import DataPreprocessorHelland
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from src.slack_alert.sofus_alert import sofus_alert
import configparser
from src.utils import get_project_root
from src.data.dataset import get_training_data
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from skmultilearn.model_selection import iterative_train_test_split, IterativeStratification
import wandb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from skopt.searchcv import BayesSearchCV
import multiprocessing
import time
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args

In [5]:
%cd src/modelling

/content/gdrive/MyDrive/Github/sports-article-classification/src/modelling


In [6]:
data = get_training_data()
data = data.loc[:, ['text', 'label']]
data_preprocessor = DataPreprocessorHelland(data['text'])
data_preprocessor.make_lower_cased()
data_preprocessor.remove_extra_spaces()
data['text'] = data_preprocessor.text_series
one_hot = MultiLabelBinarizer()
one_hot_label = one_hot.fit_transform(data['label'])
one_hot_label_list = [list(map(float, x)) for x in one_hot_label]
data['label'] = pd.Series(list(one_hot_label_list))


test_size = 0.2
X = data['text']
y = np.stack(data['label'])
stratifier = IterativeStratification(n_splits=2, order=3, sample_distribution_per_fold=[test_size, 1.0-test_size])
train_indexes, test_indexes = next(stratifier.split(X, y))

y = pd.Series(list(y), name='label')
train_split = pd.concat([X[train_indexes], y[train_indexes]], axis=1)
validation_split = pd.concat([X[test_indexes], y[test_indexes]], axis=1)

vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(train_split['text'])


In [7]:
models = [['RandomForestClassifier', RandomForestClassifier(random_state=100)],
                          ['KNeighborsClassifier', KNeighborsClassifier(metric='euclidean')],
                          ['LinearSVCOneVsRestClassifier', OneVsRestClassifier(LinearSVC(random_state=100))],
                          ['GradientBoostingClassifierMultiOutputClassifier', MultiOutputClassifier(GradientBoostingClassifier(random_state=100))],
                          ['MLPClassifier', MLPClassifier()],
                          ['AdaBoostClassifierMultiOutputClassifier', MultiOutputClassifier(AdaBoostClassifier())]]
          

hpspace_RandomForestClassifier = {
    'n_estimators': Integer(50, 500),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 0.5)
}


hpspace_KNeighborsClassifier = {
    'n_neighbors': Integer(3, 20),
    'weights': Categorical(['uniform', 'distance']),
    'algorithm': Categorical(['auto', 'kd_tree']),
    'leaf_size': Integer(10, 50),
    'p': Integer(1, 3),
}     

hpspace_LinearSVCOneVsRestClassifier = {
    'estimator__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'estimator__tol': Real(1e-6, 1e-1, prior='log-uniform'),
    'estimator__class_weight': Categorical(['balanced', None])
}


hpspace_GradientBoostingClassifierMultiOutputClassifier = {
    'estimator__n_estimators': Integer(100, 1000),
    'estimator__max_depth': Integer(3, 10),
    'estimator__min_samples_split': Integer(2, 10),
    'estimator__min_samples_leaf': Integer(1, 10),
    'estimator__max_features': Categorical(['sqrt', 'log2']),
    'estimator__learning_rate': Real(0.001, 0.1, prior='log-uniform'),
}


hpspace_MLPClassifier = {
    'hidden_layer_sizes': Integer(10, 200, prior='log-uniform'),
    'activation': Categorical(['relu', 'logistic', 'tanh']),
    'solver': Categorical(['adam', 'sgd']),
    'alpha': Real(1e-6, 1e-2, prior='log-uniform'),
    'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
    'learning_rate_init': Real(1e-4, 1e-2, prior='log-uniform'),
    'max_iter': Integer(100, 1000),
    'tol': Real(1e-6, 1e-2, prior='log-uniform')
}

hpspace_AdaBoostClassifierMultiOutputClassifier = {
    'estimator__n_estimators': Integer(10, 1000),
    'estimator__learning_rate': Real(0.001, 10, prior='log-uniform'),
    'estimator__algorithm': Categorical(['SAMME', 'SAMME.R']),
}

In [8]:

train_label_matrix = np.concatenate(train_split['label'].to_numpy()).reshape(1610 ,22)
test_label_matrix = np.concatenate(validation_split['label'].to_numpy()).reshape(399 ,22)

stratifier = IterativeStratification(n_splits=2, order=3)


counter=0

for model_name, model in models:
  if counter != 5:
    counter += 1
    continue
  else:
    counter += 1 
    print(model_name)
    params = globals()['hpspace_' + model_name]
    search = BayesSearchCV(model, params, n_iter=30, cv=stratifier,  verbose=5,
                           random_state=2, n_jobs=1, return_train_score=True)
    results = search.fit(X_vectorized, train_label_matrix)
    best_model = results.best_estimator_

    start = time.time()
    best_model.fit(X_vectorized, train_label_matrix)
    end = time.time()
    runtime = end - start
    model_config = results.cv_results_

    root = get_project_root()
    config = configparser.ConfigParser()
    config.read(root + '/src/modelling/my_config.ini')
    token_wandb = config['wandb']['TOKEN']
    wandb.login(key=token_wandb)

    model_params = model.get_params()
    model_params['model_type'] = type(model).__name__
    model_params['model_config'] = results.best_params_

    wandb.init(project='sports-article-classification-final', tags=['sklearn'], config=model_params)

    pipeline = make_pipeline(model)
    model = pipeline.fit(X_vectorized, train_label_matrix)

    pred_labels_tr = model.predict(X_vectorized)#.toarray()
    pred_labels_te = model.predict(vectorizer.transform(validation_split['text']))#.toarray()


    score_tr = model.score(X_vectorized, train_label_matrix)
    roc_auc_tr = roc_auc_score(train_label_matrix, pred_labels_tr, average='micro')
    loss_tr = log_loss(train_label_matrix, pred_labels_tr)
    score_report_tr = classification_report(train_label_matrix, pred_labels_tr, target_names=one_hot.classes_, output_dict=True)

    score_te = model.score(vectorizer.transform(validation_split['text']), test_label_matrix)
    roc_auc_te = roc_auc_score(test_label_matrix, pred_labels_te, average='micro')
    loss_te = log_loss(test_label_matrix, pred_labels_te)
    score_report_te = classification_report(test_label_matrix, pred_labels_te, target_names=one_hot.classes_, output_dict=True)


    wandb.log({'eval/accuracy': score_te, 'eval/loss': loss_te, 'eval/roc_auc': roc_auc_te,
              'eval/f1_macro': score_report_te['macro avg']['f1-score'], 'eval/f1_micro': score_report_te['micro avg']['f1-score'],
              'eval/score_report': score_report_te, 'train/train_runtime': runtime})
    wandb.finish()



AdaBoostClassifierMultiOutputClassifier
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END estimator__algorithm=SAMME.R, estimator__learning_rate=0.02228691100328716, estimator__n_estimators=786;, score=(train=0.461, test=0.280) total time= 3.9min
[CV 2/2] END estimator__algorithm=SAMME.R, estimator__learning_rate=0.02228691100328716, estimator__n_estimators=786;, score=(train=0.497, test=0.267) total time= 3.7min
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END estimator__algorithm=SAMME.R, estimator__learning_rate=2.2520889904053125, estimator__n_estimators=101;, score=(train=0.081, test=0.049) total time=  26.0s
[CV 2/2] END estimator__algorithm=SAMME.R, estimator__learning_rate=2.2520889904053125, estimator__n_estimators=101;, score=(train=0.130, test=0.075) total time=  25.6s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END estimator__algorithm=SAMME, estimator__learning_rate=0.006286343728695145, estimator__n_esti

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eirik-helland. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


eval/accuracy,▁
eval/f1_macro,▁
eval/f1_micro,▁
eval/loss,▁
eval/roc_auc,▁
train/train_runtime,▁
eval/accuracy,0.28321
eval/f1_macro,0.40109
eval/f1_micro,0.54926
eval/loss,22.30964
eval/roc_auc,0.73143


In [9]:
sofus_alert()

Sofus has sent an alert - Check slack!
